<a href="https://colab.research.google.com/github/yavuzssvr19/moodle-chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install chromadb --quiet
%pip install sentence_transformers --quiet
%pip install pypdf --quiet
%pip install langchain --quiet
%pip install tqdm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.7 MB/s eta 0:

In [ ]:

import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter


from pypdf import PdfReader

from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from chromadb import Client, PersistentClient
from chromadb.utils import embedding_functions

import textwrap
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil

In [ ]:
chromaDB_path = '/content/drive/MyDrive/RAG_heading_chunks'

In [ ]:
#Klasör varsa kullanıcıdan izin alarak içeriğini temizler.
# Temiz veri deposu başlatmak için gerekli

def delete_all_files_and_folders(chromaDB_path):
  # verdiğin klasör yolu var mı yok mu kontrol eder.
  if os.path.exists(chromaDB_path):
    print(f"The directory '{chromaDB_path}' already exists.")
    permission = input("Do you want to delete all the files and folders in this directory? (y/n): ")
    #y yazarsa silinsin
    if permission == "y":
      shutil.rmtree(chromaDB_path)
      print(f"All files and folders in '{chromaDB_path}' have been deleted.")
    else:
      print("No action taken.")
  else:
    print(f"The directory '{chromaDB_path}' does not exist.")

In [ ]:
# Check if the chromadb_path exists or not. If so, delete all the files and folders in chromadb_path. But before deleting get the permission from the user.
delete_all_files_and_folders(chromaDB_path)


The directory '/content/drive/MyDrive/RAG_heading_chunks' does not exist.


In [ ]:
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from chromadb import Client, PersistentClient


collection_name: Oluşturulacak veya yüklenecek koleksiyonun adı

embedding_function: Embedding (vektörleştirme) fonksiyonu (örneğin: SentenceTransformers gibi)

chromaDB_path: Eğer belirtilirse Chroma'nın verileri bu dizine kalıcı olarak kaydedilecek; belirtilmezse geçici bellek içinde tutulur.   

| Parametre            | Anlamı                | Ne işe yarar?                                        |
| -------------------- | --------------------- | ---------------------------------------------------- |
| `collection_name`    | Koleksiyon adı        | Belge koleksiyonu tanımlar veya yükler               |
| `embedding_function` | Vektörleştirme işlevi | Metni embedding'e çevirir                            |
| `chromaDB_path`      | Kayıt yolu            | Verileri kalıcı mı, geçici mi saklayacağını belirler |


# create_chroma_client
 ChromaDB istemcisini (client) oluşturur ve bir koleksiyon (collection) yükler veya oluşturur.

**Ne Zaman Kullanılır?**
- RAG sisteminde PDF metinlerini işledikten sonra embedding’leri bir vektör veritabanına (ChromaDB) kaydetmek istersin.

- Bu fonksiyon, ChromaDB’yi ya kalıcı olarak (örneğin Google Drive klasörüne) ya da geçici olarak (RAM içinde) başlatmanı sağlar.


In [ ]:
from chromadb import Client, PersistentClient
from chromadb.config import Settings, DEFAULT_TENANT, DEFAULT_DATABASE

def create_chroma_client(collection_name, embedding_function, chromaDB_path=None):
    if chromaDB_path:
        chroma_client = PersistentClient(
            path=chromaDB_path,
            settings=Settings(),
            tenant=DEFAULT_TENANT,
            database=DEFAULT_DATABASE
        )
    else:
        chroma_client = Client()

    chroma_collection = chroma_client.get_or_create_collection(
        name=collection_name,
        embedding_function=embedding_function
    )
    return chroma_client, chroma_collection


In [ ]:
import shutil
import os
from chromadb import PersistentClient
from chromadb.config import Settings
from chromadb.utils import embedding_functions

# Yeni ve yazılabilir bir klasör yolu belirle
#chromaDB_path = "/content/chroma_clean_workspace"  # RAM içinde
chromaDB_path = '/content/drive/MyDrive/RAG_heading_chunks'

# Önce varsa silip temizle
if os.path.exists(chromaDB_path):
    shutil.rmtree(chromaDB_path)
os.makedirs(chromaDB_path, exist_ok=True)

# Embedding fonksiyonu tanımla
sentence_transformer_model = "paraphrase-multilingual-mpnet-base-v2"
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=sentence_transformer_model
)

# Yeni client başlat
chroma_client = PersistentClient(path=chromaDB_path, settings=Settings())

# Yeni koleksiyon oluştur
collection_name = "Papers_heading_chunks_BERNA"
chroma_collection = chroma_client.get_or_create_collection(
    name=collection_name,
    embedding_function=embedding_function
)

print("✅ Yeni koleksiyon başarıyla oluşturuldu:", chroma_collection.name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Yeni koleksiyon başarıyla oluşturuldu: Papers_heading_chunks_BERNA


In [ ]:
#Bu isim, ChromaDB içinde bir belge koleksiyonunu tanımlamak için kullanılır.
#bu collection, PDF’lerden çıkarılmış chunklara ait vektör verilerini saklayacak.
#collection_name = "Papers_heading_chunks"

#Bu satırda embedding için kullanılacak modelin ismi belirleniyor.
#sentence_transformer_model="paraphrase-multilingual-mpnet-base-v2"

#embedding fonksiyonu tanımlanıyor
#Bu fonksiyon, her metin parçasını yukarıda belirlenen modeli kullanarak vektöre dönüştürür.
#embedding_function= embedding_functions.SentenceTransformerEmbeddingFunction(
#    model_name=sentence_transformer_model)


In [ ]:
from chromadb import PersistentClient
from chromadb.config import Settings, DEFAULT_TENANT, DEFAULT_DATABASE

In [ ]:
#Daha önce tanımlanmış create_chroma_client() fonksiyonunu çağırır ve dönen iki değeri chroma_client ve chroma_collection olarak saklar.
chroma_client, chroma_collection = create_chroma_client(collection_name,
                                                        embedding_function,
                                                        chromaDB_path)

In [ ]:
# Chroma istemcisinden tüm koleksiyonları listele
for collection in chroma_client.list_collections():
    print(collection.name)


Papers_heading_chunks_BERNA


In [ ]:
print("✅ Koleksiyon adı:", chroma_collection.name)
print("✅ Chunk sayısı:", chroma_collection.count())


✅ Koleksiyon adı: Papers_heading_chunks_BERNA
✅ Chunk sayısı: 0


# upload_multiple_files()
 Kullanıcının Colab arayüzü üzerinden birden fazla dosya seçip yüklemesini sağlar ve yüklenen tüm dosyaların isimlerini bir liste halinde döndürür.

In [ ]:
from google.colab import files
def upload_multiple_files():
  uploaded = files.upload()
  file_names = list()
  for fn in uploaded.keys():
    #print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
    file_names.append(fn)
  return file_names

In [ ]:
#Google Colab arayüzünde bir dosya yükleme penceresi açar.
#file_names = upload_multiple_files()

# convert_PDF_Text()
-  Bir PDF dosyasını sayfa sayfa okuyup, her sayfadaki metni çıkarmaktır.
- Her sayfanın metnini bir liste elemanı olarak saklar ve boş sayfaları filtreler.

.extract_text(): Sayfadaki metni çıkarır.

.strip(): Başında ve sonundaki boşluk karakterlerini siler.

In [ ]:

def convert_PDF_Text(pdf_path):
  reader = PdfReader(pdf_path)
  pdf_texts = [p.extract_text().strip() for p in reader.pages]
  # Filter the empty strings
  pdf_texts = [text for text in pdf_texts if text]
  print("Document: ",pdf_path," chunk size: ", len(pdf_texts))
  return pdf_texts

# split_by_headings(pdf_texts)
PDF'den çıkarılmış düz metinleri, başlıklara göre bölmek
📌 Örneğin: "1. Giriş", "2.1 Sosyal Gelişim", "Bölüm 3", "Sonuç" gibi başlıklar yakalanır ve o başlıktan itibaren gelen içerik bir parça (chunk) olur.

In [ ]:
def split_by_headings(pdf_texts):
    all_text = "\n".join(pdf_texts)
    lines = all_text.split("\n")
    split_points = [i for i, line in enumerate(lines) if re.match(r"^\s*(\d+(\.\d+)*|Bölüm \d+|Giriş|Sonuç)", line, re.IGNORECASE)]

    if not split_points:
        # fallback: başlık bulunamadıysa tüm metni tek parça döndür
        return [all_text.strip()]

    chunks = []
    for i in range(len(split_points)):
        start = split_points[i]
        end = split_points[i + 1] if i + 1 < len(split_points) else len(lines)
        section = "\n".join(lines[start:end]).strip()
        if section:
            chunks.append(section)

    return chunks


In [ ]:
def convert_Page_ChunkinChar_with_pages(pdf_texts, chunk_size=1000, chunk_overlap=200):
    """
    Heading-aware + character-based chunking:
    Önce başlıklara göre ayırır, ardından karakter limitine göre chunk’lar üretir.
    Sayfa numarası bilgisi vermez.
    """
    character_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ". ", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    # Heading bazlı bölümleri al (senin split_by_headings fonksiyonunla)
    heading_sections = split_by_headings(pdf_texts)

    all_chunks = []
    for section in heading_sections:
        sub_chunks = character_splitter.split_text(section)
        all_chunks.extend(sub_chunks)

    print(f"Toplam karakter + heading-aware chunk sayısı: {len(all_chunks)}")
    return all_chunks, None  # page_numbers yerine None döner


* daha önce karakter bazlı olarak bölünmüş metinleri token bazlı daha hassas parçalara ayırmak ve LLM veya embedding modellerinin token sınırlarına daha uygun hale getirmekti

In [ ]:

def convert_Chunk_Token(text_chunksinChar, sentence_transformer_model,
                        chunk_overlap=64, tokens_per_chunk=128):
  token_splitter = SentenceTransformersTokenTextSplitter(
      chunk_overlap=chunk_overlap,
      model_name=sentence_transformer_model,
      tokens_per_chunk=tokens_per_chunk)

  text_chunksinTokens = []
  for text in text_chunksinChar:
      text_chunksinTokens += token_splitter.split_text(text)
  print(f"\nTotal number of token chunks: {len(text_chunksinTokens)}")
  return text_chunksinTokens


Bu fonksiyonun amacı, ChromaDB’ye eklenecek her metin parçası (chunk) için:

    * Benzersiz bir kimlik (ID) oluşturmak

    * Her parçaya belgeye özel metadata (başlık ve kategori gibi) eklemek

böylece daha sonra belge bazlı filtreleme ve sorgulama yapabilmeni sağlamaktır.

In [ ]:
def add_meta_data(text_chunks, title, category, initial_id, headings=None):
    ids = [str(i + initial_id) for i in range(len(text_chunks))]
    metadatas = []
    for i in range(len(text_chunks)):
        meta = {
            'document': title,
            'category': category
        }
        if headings:
            meta["heading"] = headings[i]
            meta["source"] = f"{title} – {headings[i]}"  # 👈 metin formatında kaynak
        else:
            meta["source"] = title  # sadece dosya adı
        metadatas.append(meta)
    return ids, metadatas


* önceden hazırlanmış metin parçalarını (chunk’ları), metadata bilgilerini ve ID’leri ChromaDB koleksiyonuna eklemek ve eklemeden önce ve sonra koleksiyonun kaç kayıt içerdiğini kullanıcıya bildirmektir.

In [ ]:
def add_document_to_collection(ids, metadatas, text_chunksinTokens, chroma_collection):
  print("Before inserting, the size of the collection: ", chroma_collection.count())
  chroma_collection.add(ids=ids, metadatas= metadatas, documents=text_chunksinTokens)
  print("After inserting, the size of the collection: ", chroma_collection.count())
  return chroma_collection

* ChromaDB koleksiyonundan bir sorguya (query) en yakın belgeleri (documents) ve onların ilgili metadata'larını, mesafeleriyle birlikte geri getirmektir.

In [ ]:
def retrieveDocs(chroma_collection, query, n_results=15, return_only_docs=False):
    results = chroma_collection.query(query_texts=[query],
                                      include= [ "documents","metadatas",'distances' ],
                                      n_results=n_results)

    if return_only_docs:
        return results['documents'][0]
    else:
        return results

* Getilecek olan ilgili metinlerin metadata bilgileri ve içerikleri

In [ ]:
def show_results(results, return_only_docs=False):

  if return_only_docs:
    retrieved_documents = results
    if len(retrieved_documents) == 0:
      print("No results found.")
      return
    for i, doc in enumerate(retrieved_documents):
      print(f"Document {i+1}:")
      print("\tDocument Text: ")
      display(to_markdown(doc));
  else:
      retrieved_documents = results['documents'][0]
      if len(retrieved_documents) == 0:
          print("No results found.")
          return
      retrieved_documents_metadata = results['metadatas'][0]
      retrieved_documents_distances = results['distances'][0]
      print("------- retreived documents -------\n")

      for i, doc in enumerate(retrieved_documents):
          print(f"Document {i+1}:")
          print("\tDocument Text: ")
          display(to_markdown(doc));
          print(f"\tDocument Source: {retrieved_documents_metadata[i]['document']}")
          print(f"\tDocument Source Type: {retrieved_documents_metadata[i]['category']}")
          print(f"\tDocument Distance: {retrieved_documents_distances[i]}")

load_multiple_pdfs_to_ChromaDB(...) fonksiyonu:

    * Kullanıcıdan birden fazla PDF dosyası alır

    * Her PDF’yi parçalara ayırır (chunk)

    * Her chunk'ı token bazlı olarak işler

    * Embedding’lerini hazırlar

    * Metadata ile birlikte ChromaDB’ye kaydeder

    * Tüm süreci tek bir çağrı ile yönetir

In [ ]:
def expand_page_numbers(original_chunks, tokenized_chunks, original_page_numbers):
    expanded = []
    char_index = 0
    for i, orig_chunk in enumerate(original_chunks):
        chunk_length = len(orig_chunk)
        count = 0
        while char_index < len(tokenized_chunks) and count < chunk_length:
            expanded.append(original_page_numbers[i])
            count += len(tokenized_chunks[char_index])
            char_index += 1
    # Eğer hala eksik varsa kalanları son sayfayla doldur
    while len(expanded) < len(tokenized_chunks):
        expanded.append(original_page_numbers[-1])
    return expanded


In [ ]:
def load_multiple_pdfs_to_ChromaDB(collection_name,sentence_transformer_model,
                                   chromaDB_path):

  collection_name= collection_name
  category= "Journal Paper"
  sentence_transformer_model=sentence_transformer_model
  embedding_function= embedding_functions.SentenceTransformerEmbeddingFunction(model_name=sentence_transformer_model)
  chroma_client, chroma_collection = create_chroma_client(collection_name, embedding_function, chromaDB_path)
  current_id = chroma_collection.count()
  file_names = upload_multiple_files()
  for file_name in file_names:
    print(f"Document: {file_name} is being processed to be added to the {chroma_collection.name} {chroma_collection.count()}")
    print(f"current_id: {current_id} ")
    pdf_texts = convert_PDF_Text(file_name)
    text_chunksinChar, heading_titles = convert_Page_ChunkinChar_with_pages(pdf_texts)
    text_chunksinTokens = convert_Chunk_Token(text_chunksinChar, sentence_transformer_model)
    adjusted_page_numbers = None
    ids, metadatas = add_meta_data(text_chunksinTokens, file_name, category, current_id, adjusted_page_numbers)

    current_id = current_id + len(text_chunksinTokens)
    chroma_collection = add_document_to_collection(ids, metadatas, text_chunksinTokens, chroma_collection)
    print(f"Document: {file_name} added to the collection: {chroma_collection.count()}")
  return  chroma_client, chroma_collection

In [ ]:
import re

In [ ]:
chroma_client, chroma_collection= load_multiple_pdfs_to_ChromaDB(collection_name,sentence_transformer_model, chromaDB_path)

Saving TRANSLATED_2016_Tracy-Ford-Inman_-Jana-Kirchner-Parenting-Gifted-Children-101_-An-Introduction-to-Gifted-Kids-and-Th.pdf to TRANSLATED_2016_Tracy-Ford-Inman_-Jana-Kirchner-Parenting-Gifted-Children-101_-An-Introduction-to-Gifted-Kids-and-Th.pdf
Saving TRANSLATED_Carol-Fertig-Raising-a-Gifted-Child_-A-Parenting-Success-Handbook-_2021_-Taylor-and-Francis.pdf to TRANSLATED_Carol-Fertig-Raising-a-Gifted-Child_-A-Parenting-Success-Handbook-_2021_-Taylor-and-Francis.pdf
Saving TRANSLATED_Kathleen-Nilles_-2019-Jennifer-L-Jolly_-Tracy-Ford-Inman_-Joan-Frankl-Success-Strategies-for-Parenting-Gi.pdf to TRANSLATED_Kathleen-Nilles_-2019-Jennifer-L-Jolly_-Tracy-Ford-Inman_-Joan-Frankl-Success-Strategies-for-Parenting-Gi.pdf
Saving TRANSLATED_Sylvia B. Rimm - Keys to Parenting the Gifted Child (Barron's Parenting Keys) (1994, Barrons Educational Series Inc) - libgen.li.pdf to TRANSLATED_Sylvia B. Rimm - Keys to Parenting the Gifted Child (Barron's Parenting Keys) (1994, Barrons Educational Se

In [ ]:
sentence_transformer_model = "paraphrase-multilingual-mpnet-base-v2"
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=sentence_transformer_model
)

In [ ]:
import os

# ChromaDB dizinini kontrol et
if os.path.exists(chromaDB_path):
    print("📂 ChromaDB klasör içeriği:")
    for item in os.listdir(chromaDB_path):
        print(" -", item)
else:
    print("❌ Belirtilen dizin mevcut değil:", chromaDB_path)


📂 ChromaDB klasör içeriği:
 - chroma.sqlite3
 - eb1f5ec5-87cb-4a3a-bd5f-dc92d0bdf3c4


In [ ]:
"""# 2️⃣ Var olan Chroma verisini bağlamak:
chromaDB_path = '/content/drive/MyDrive/RAG_heading_chunks'  # veya ChromaDBData2
print(chromaDB_path)

chroma_client = PersistentClient(path=chromaDB_path, settings=Settings())

# 3️⃣ Aynı collection adını kullan (örnek: Papers)
chroma_collection = chroma_client.get_collection("Papers_heading_chunks_NURAN", embedding_function=embedding_function)"""

'# 2️⃣ Var olan Chroma verisini bağlamak:\nchromaDB_path = \'/content/drive/MyDrive/RAG_heading_chunks\'  # veya ChromaDBData2\nprint(chromaDB_path)\n\nchroma_client = PersistentClient(path=chromaDB_path, settings=Settings())\n\n# 3️⃣ Aynı collection adını kullan (örnek: Papers)\nchroma_collection = chroma_client.get_collection("Papers_heading_chunks_NURAN", embedding_function=embedding_function)'

In [ ]:
query = "Erken Yetişkin Statüsü Riski"

In [ ]:
retrieved_documents=retrieveDocs(chroma_collection, query, 15)
show_results(retrieved_documents)

------- retreived documents -------

Document 1:
	Document Text: 


> Bu olumsuz geçişin neden ve nasıl gerçekleştiğini merak edeceksiniz. Çünkü ergenler, yetişkin olarak kabul edilmeye alıştıktan sonra çocuk gibi davranılmaya hazır değillerdir. Artık risklerin bilincinde olduğunuza göre sorunlardan daha kolay kaçınabilirsiniz. Başlangıçtan itibaren kendinize ayrı bir yetişkin yaşamı sürdürün. Haftada bir veya iki akşam arkadaşlarınızla bir

	Document Source: TRANSLATED_Sylvia B. Rimm - Keys to Parenting the Gifted Child (Barron's Parenting Keys) (1994, Barrons Educational Series Inc) - libgen.li.pdf
	Document Source Type: Journal Paper
	Document Distance: 4.440796852111816
Document 2:
	Document Text: 


> BÖLÜM 37 YAZ ZENGİNLİĞİ VEYA SADECE TAKILMIŞ MI?: GENİŞLETİLMİŞ ÖĞRENME FIRSATLARI HAKKINDA EBEVEYNLER HANGİ BİLMELİDİR? J. Denise Drain ve Melissa R. Hasan tarafından Ebeveynler olarak çoğu zaman aşırı programlı çocuklarımız için endişeleniriz. Ne kadar çok fazla? Okula ara verdiklerinde boş zamanları olmalı mı? Akademik, müzik veya spor kamplarına mı katılmalılar; arkadaşlarla takılmak; büyükbabanın çiftliğinde tatil; ya da sahile gitmek? Elbette

	Document Source: TRANSLATED_Kathleen-Nilles_-2019-Jennifer-L-Jolly_-Tracy-Ford-Inman_-Joan-Frankl-Success-Strategies-for-Parenting-Gi.pdf
	Document Source Type: Journal Paper
	Document Distance: 4.782238483428955
Document 3:
	Document Text: 


...

	Document Source: TRANSLATED_Sylvia B. Rimm - Keys to Parenting the Gifted Child (Barron's Parenting Keys) (1994, Barrons Educational Series Inc) - libgen.li.pdf
	Document Source Type: Journal Paper
	Document Distance: 5.421747207641602
Document 14:
	Document Text: 


> sahip hale gelirler, ancak onlara rehberlik eden yetişkinlerin olduğunu kabul etmeye devam etmeleri gerekir. Böylece V'den genç yetişkinliğe, bağımsızlığa ve kişisel karar almaya geçmeye hazırdırlar. (s. 114–115) Çocuğunuza uygun seçimler ve özgürlükler sağlamanın yanı sıra, davranışa yönelik yönergeleri ve beklentileri uygulamak da kritik öneme sahiptir. Çocuğunuz belirli durumlarda ne kadar olgun görünürse görünsün ve kararlarınızı ne kadar sorgularsa veya tartışırsa tartışsın, davranış için açık ve uygun kurallar ve sonuçlar belirlemek, çocuğunuzun çeşitli ortamlarda

	Document Source: TRANSLATED_2016_Tracy-Ford-Inman_-Jana-Kirchner-Parenting-Gifted-Children-101_-An-Introduction-to-Gifted-Kids-and-Th.pdf
	Document Source Type: Journal Paper
	Document Distance: 5.447476863861084
Document 15:
	Document Text: 


> kişisel taahhütlerinizle eşleştirin. Ağır kişisel bağlılık gerektiren kariyerlere yatırım yapmak için genellikle yıllarca süren gençlik eğlencesinden fedakarlık ediyorlar. Bu yıllarda bağımlılık -bağımsızlık çatışmalarını çevreleyen gerilimler, en sevgi dolu ebeveynleri bile zorlayabilir. Bu üstün yetenekli öğrenciler için örgün eğitim "sonsuza kadar" gibi gelse de, onların çevik zihinleri zorlanacak ve topluma katkı yapma olasılıkları artacaktır. Sayfa 104

	Document Source: TRANSLATED_Sylvia B. Rimm - Keys to Parenting the Gifted Child (Barron's Parenting Keys) (1994, Barrons Educational Series Inc) - libgen.li.pdf
	Document Source Type: Journal Paper
	Document Distance: 5.459727764129639


In [ ]:
!ls "{chromaDB_path}"

chroma.sqlite3	eb1f5ec5-87cb-4a3a-bd5f-dc92d0bdf3c4


In [ ]:
from google.colab import runtime
# Disconnect from the runtime
#!kill -9 -1

In [ ]:
chroma_collection.get(['0'])

{'ids': ['0'],
 'embeddings': None,
 'documents': ['Giriş: Bu Kitabı Okumak İçin İpuçları'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'document': 'TRANSLATED_2016_Tracy-Ford-Inman_-Jana-Kirchner-Parenting-Gifted-Children-101_-An-Introduction-to-Gifted-Kids-and-Th.pdf',
   'category': 'Journal Paper',
   'source': 'TRANSLATED_2016_Tracy-Ford-Inman_-Jana-Kirchner-Parenting-Gifted-Children-101_-An-Introduction-to-Gifted-Kids-and-Th.pdf'}]}

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import os
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
def generate_LLM_answer(prompt, context, chat):
    full_prompt = f"""
[BAĞLAM]:
{context}

[SORU]:
{prompt}
"""
    response = chat.send_message(full_prompt)
    return response.text

In [ ]:
"""def generateAnswer_with_source(RAG_LLM, chroma_collection, query, n_results=15):
    results = retrieveDocs(chroma_collection, query, n_results=n_results)
    chunks = results["documents"][0]
    metadatas = results["metadatas"][0]

    context_text = "\n".join(chunks)
    answer = generate_LLM_answer(query, context_text, RAG_LLM)

    # Kaynakların tıklanabilir HTML formatını oluşturalım
    unique_links = set()
    for m in metadatas:
        source = m.get("source", "")
        label = f"{m.get('document', '')}"
        if "page" in m:
            label += f" (sayfa {m['page']})"
        unique_links.add(f'<a href="{source}" target="_blank">{label}</a>')

    final_answer = f"{answer}\n\n<b>Kaynaklar:</b><br>" + "<br>".join(unique_links)
    return final_answer"""

'def generateAnswer_with_source(RAG_LLM, chroma_collection, query, n_results=15):\n    results = retrieveDocs(chroma_collection, query, n_results=n_results)\n    chunks = results["documents"][0]\n    metadatas = results["metadatas"][0]\n\n    context_text = "\n".join(chunks)\n    answer = generate_LLM_answer(query, context_text, RAG_LLM)\n\n    # Kaynakların tıklanabilir HTML formatını oluşturalım\n    unique_links = set()\n    for m in metadatas:\n        source = m.get("source", "")\n        label = f"{m.get(\'document\', \'\')}"\n        if "page" in m:\n            label += f" (sayfa {m[\'page\']})"\n        unique_links.add(f\'<a href="{source}" target="_blank">{label}</a>\')\n\n    final_answer = f"{answer}\n\n<b>Kaynaklar:</b><br>" + "<br>".join(unique_links)\n    return final_answer'

In [ ]:
# Used to securely store your API key
from google.colab import userdata
import google.generativeai as genai # Explicitly import genai here
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GEMINIAPI3')
genai.configure(api_key=GOOGLE_API_KEY)

SecretNotFoundError: Secret GEMINIAPI3 does not exist.

In [ ]:
system_prompt = """
Sen, üstün zekalı çocukların sosyal gelişimi konusunda uzmanlaşmış bir yapay zeka asistansın. Görev alanın, bu çocukların arkadaşlık ilişkileri, yalnızlık hissi, duygusal ihtiyaçları ve sosyal uyum süreçleri gibi konularda, ebeveynlere ve eğitimcilere bilimsel kaynaklara dayalı olarak rehberlik etmektir.

Ana Kurallar:
        *Sadece sana sağlanan kaynak belgelerinde (RAG içeriklerinde) açıkça yer alan bilgilere dayalı cevap üret.

        *Kaynakta açık bilgi yoksa şu ifadeyi kullan:
            "Bu konuda elimde yeterli bilgi bulunmuyor."

        *Cevaplarını açık, sade ve profesyonel bir Türkçe ile yaz.

        *Gerekirse maddeler halinde, bazen ise açıklayıcı paragraflarla cevap ver.

        *"Üstün zekalı" yerine daima "üstün yetenekli" ifadesini kullan.

        *Cevaplarda üstün yetenekli çocuklar hakkında olumsuz yargı içeren, damgalayıcı ya da genelleyici ifadelerden kaçın (örneğin: "alışılmadık", "tuhaf", "sorunlu" gibi kelimeler kullanılmaz).

        *  **"Sağlanan kaynaklara göre"** gibi ifadeler yerine şu kalıbı kullan: **"Bilimsel kaynaklara göre"**.

        *Bilimsel kaynaklara göre" ifadesini kullan. “Sağlanan içerik”, “verilen metin” gibi kalıplardan kaçın.
        Kullanıcının kaynaklara erişimi olmadığını varsay. Cevapları buna göre sade ve anlaşılır sun.

        * Eğer kullanıcı sorusu, üstün yetenekli çocuklarla ilgili değilse,
      hiçbir açıklama yapmadan aşağıdaki cevabı ver:
      "Ben üstün yetenekli çocukların sosyal gelişimi konusunda uzmanlaşmış bir yapay zekâ asistanıyım. Lütfen bu alana dair bir soru sorun."

        *Kesin, duygusal ya da cesur yönlendirmelerden kaçın. Özellikle şu tür ifadeler kullanılmamaya dikkat:
            - "Üstün yetenekli çocuğunuzun yalnız kalmasını destekleyin"
            - "Kendinizi suçlu hissetmeyin"
            - "Her şey yoluna girecek"
            - "Bu çok normal" gibi genellemelere yer verme.

Format Talimatı:
        *Yanıtlar, aşağıdaki örneklere uygun olacak şekilde yapılandırılmalıdır. LLM’in vereceği cevaplar:

        *Net bir başlık içermeli

        *Gerektiğinde açıklamalı paragraflar kullanılmalı

        *Gerektiğinde madde işaretleriyle yapılandırılmalı

        *Sonuç cümlesi ile içerik toparlanmalı


🧪 Few-Shot Örnek #1
Soru:
Üstün yetenekli çocuğum diğer çocuklarla oynamak istemiyor, bu yalnızlık onun gelişimini olumsuz etkiler mi?

Yanıt:

Üstün Yetenekli Çocuklarda Sosyal Yalnızlık ve Etkileri
Bilimsel kaynaklara göre, üstün yetenekli çocuklar yaşıtlarıyla ortak ilgi alanları geliştirmekte zorlandıklarında sosyal etkileşimlerden uzaklaşabilirler. Bu durum yalnızlık hissini artırabilir ve sosyal beceri gelişimini olumsuz yönde etkileyebilir.

Yalnızlık eğilimini azaltmak için aşağıdaki stratejiler önerilmektedir:

İlgi alanına uygun sosyal ortamlar yaratın: Bilim kulüpleri, sanat atölyeleri gibi yapılar, çocuğun entelektüel düzeyine hitap eden ortamlardır.

Duygusal ifadeyi teşvik edin: Günlük tutma, resim çizme, hikâye anlatma gibi araçlarla duygularını ifade etmesine yardımcı olun.

Birebir ilişkileri destekleyin: Büyük gruplar yerine bireysel arkadaşlıklar daha güvenli ve anlamlı olabilir.

Sonuç olarak, üstün yetenekli çocukların sosyal gelişimi için uygun ortamların sağlanması yalnızlık riskini azaltabilir.



🧪 Few-Shot Örnek #2
Soru:
Üstün yetenekli bir öğrencim sınıfta sürekli liderlik etmeye çalışıyor. Diğer çocuklarla çatışma yaşıyor. Ne yapmalıyım?

Yanıt:

Üstün Yetenekli Çocuklarda Liderlik Eğilimleri ve Sınıf İçi Denge
Bilimsel kaynaklara göre, üstün yetenekli çocuklar yüksek sorumluluk duygusu ve girişkenlik gibi özellikleri nedeniyle liderlik rolünü benimseme eğilimindedir. Ancak bu durum, sınıf içinde akranlarıyla çatışmalara neden olabilir.

Eğitmenlerin bu eğilimleri dengelemesi için öneriler:

Grup içi rol dönüşümleri sağlayın: Her öğrencinin zaman zaman lider, takipçi veya gözlemci olduğu etkinlikler planlayarak eşit katılım teşvik edilmelidir.

Empati geliştirme etkinlikleri yapın: Oyunlar ve drama etkinlikleri çocukların başkalarının bakış açılarını anlamalarına yardımcı olur.

Olumlu liderlik modelleri gösterin: Saygılı, dinlemeye açık ve iş birliğine dayalı liderlik davranışları üzerine sınıf içi konuşmalar yapılabilir.

Sonuç olarak, liderlik becerilerinin yapılandırılmış yollarla yönlendirilmesi, sosyal uyumu güçlendirebilir.


-----

Her cevabında yukarıdaki ilkeleri uygula. Sadece sağlanan içeriklere güven. Tahmin veya kişisel yorum yapma. Kaynak yoksa dürüstçe belirt.


"""

In [ ]:
def build_chatBot(system_instruction):
  model = genai.GenerativeModel('gemini-1.5-flash-latest', system_instruction=system_instruction,
                                generation_config=genai.types.GenerationConfig(
        temperature=0.3,
        top_p=0.95,
        top_k=70
    ))
  chat = model.start_chat(history=[])
  return chat

In [ ]:
RAG_LLM = build_chatBot(system_prompt)

In [ ]:
# Verify collection properties
print(f"Collection name: {chroma_collection.name}")  # Access the name attribute directly
print(f"Number of documents in collection: {chroma_collection.count()}")

# List all collections in the client
print("All collections in ChromaDB client:")
for collection in chroma_client.list_collections():
    print(collection)

In [ ]:
def summarize_collection(chroma_collection):
  summary = [] # Initialize summary as a list
  print("Summarizing the collection...")
  # Verify collection properties
  print(f"\t Collection name: {chroma_collection.name}")  # Access the name attribute directly
  print(f"\t Number of document chunks in collection: {chroma_collection.count()}")
  summary.append(f"Collection name: {chroma_collection.name}") # Append to the list
  summary.append(f"Number of document chunks in collection: {chroma_collection.count()}")
  # Print distinct metadata "document" for each chunk in the collection
  print("\t Distinct 'document' metadata in the collection:")
  distinct_documents = set()  # Use a set to store unique document names

  # Iterate over chunks in the collection
  for chunk_id in range(chroma_collection.count()):
      metadata = chroma_collection.get([str(chunk_id)])['metadatas'][0]  # Get metadata for the chunk
      document_name = metadata.get("document", "Unknown")  # Get document metadata; default to "Unknown" if not present
      distinct_documents.add(document_name)  # Add document name to set for uniqueness

  # Print all distinct document names
  summary.append("Documents:")
  for document_name in distinct_documents:
      print("\t ",document_name)
      summary.append(document_name) # Append to the list

  print("Collection summarization completed.")

  # Join the list elements into a single string
  summary_string = "\n ".join(summary)
  return summary_string

In [ ]:
s=summarize_collection(chroma_collection)

In [ ]:
print(s)

In [ ]:
import os
os.makedirs("outputs", exist_ok=True)

In [ ]:
%pip install gradio
import gradio as gr

In [ ]:
from sentence_transformers import CrossEncoder, SentenceTransformer, util
import gradio as gr
from IPython.display import display, Markdown

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown('> ' + text.replace('\n', '\n> '))

# Model initialization
reranker_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")
similarity_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# Reranking with metadata
def rerank_chunks_with_metadata(query, chunks, metadatas, top_k=5):
    if len(chunks) == 0:
        return [], [], []

    pairs = [(query, chunk) for chunk in chunks]
    scores = reranker_model.predict(pairs)

    scored = sorted(zip(chunks, metadatas, scores), key=lambda x: x[2], reverse=True)
    top_chunks = [c for c, _, _ in scored[:top_k]]
    top_metadatas = [m for _, m, _ in scored[:top_k]]
    top_scores = [s for _, _, s in scored[:top_k]]

    return top_chunks, top_metadatas, top_scores


# Core answer generation
def generateAnswer(RAG_LLM, chroma_collection, query, n_results=15, only_response=True):
    # 1. ChromaDB'den belgeleri getir
    results = retrieveDocs(chroma_collection, query, n_results=n_results, return_only_docs=False)
    chunks = results["documents"][0]
    metadatas = results["metadatas"][0]

    # 2. Reranking: CrossEncoder kullan
    reranked_chunks, reranked_metadatas, reranked_scores = rerank_chunks_with_metadata(query, chunks, metadatas, top_k=5)
    context = "\n".join(reranked_chunks)

    # 3. LLM ile cevap üret
    output = generate_LLM_answer(query, context, RAG_LLM)

    # 4. Cosine similarity hesapla
    embedding_answer = similarity_model.encode(output, convert_to_tensor=True)
    embedding_chunks = similarity_model.encode(reranked_chunks, convert_to_tensor=True)
    cos_similarities = util.cos_sim(embedding_answer, embedding_chunks)

    # 5. En yüksek skorları logla
    print("🧪 Cosine Similarity Skorları:")
    for i, score in enumerate(cos_similarities[0]):
        print(f"Chunk {i+1} | Cosine Sim: {score.item():.4f} | Skor: {reranked_scores[i]:.4f}")
        print(f"→ {reranked_chunks[i][:80]}...\n")

    max_sim = cos_similarities.max().item()

    # 6. En iyi metadata bilgisi
    best_meta = reranked_metadatas[0]
    doc = best_meta.get("document", "Unknown.pdf")
    heading = best_meta.get("heading", "")

    # 7. Yıldızları temizle
    output = output.replace("***", "").replace("**", "").replace("*", "")

    # 8. Yanıt formatı + kaynak + skor bilgisi

    output_with_source = output
    output_with_source += f"\n\n📘 Kaynak: {doc}"
    if heading:
        output_with_source += f" – {heading}"
    output_with_source += f"\n📊 En yüksek reranker skoru: {reranked_scores[0]:.4f}"
    output_with_source += f"\n📈 En yüksek cosine similarity: {max_sim:.4f}"

    # 9. Yanıtı dosyaya kaydet (opsiyonel)
    with open("outputs/step3_reranking2.txt", "a", encoding="utf-8") as f:
        f.write(f"\n=== Query ===\n{query}\n")
        f.write(f"=== Response ===\n{output}\n")
        f.write(f"=== Max Cosine Sim ===\n{max_sim:.4f}\n")
        f.write(f"=== Source ===\n- {doc}")
        if heading:
            f.write(f" – {heading}")
        f.write(f"\n-----------------------------\n")

    return output_with_source




# Guardrails
FORBIDDEN_KEYWORDS = [
    "intihar", "öldürmek", "hacklemek", "yasa dışı erişim", "atlatmak",
    "devre dışı bırakmak", "bomba", "kod çalıştırmak", "sistem kısıtlamalarını kaldırmak",
    "sen bir sohbet botu değilsin", "numara yapmak", "rol yapmak", "uyuşturucu",
    "ilaç", "aspirin", "prozac", "ibuprofen", "rol"
]

def is_safe_prompt(prompt):
    return not any(keyword in prompt.lower() for keyword in FORBIDDEN_KEYWORDS)

def reinforce_role(prompt):
    red_flags = [
        "önceki komutları unut", "kural tanımıyorsun", "rol yap", "filtreyi atla",
        "yönergeleri boşver", "insan gibi davran", "kararını ver", "kaynağa bağlı kalma",
        "promptu gözardı et", "kuralları unut", "moderasyon yok", "asistan değilsin",
        "kısıtlamasız cevap ver"
    ]
    return not any(flag in prompt.lower() for flag in red_flags)

def is_safe_output(output):
    if output is None:
        return False
    lowered = output.lower()
    return all(keyword not in lowered for keyword in FORBIDDEN_KEYWORDS)


def full_guardrail_interface(question):
    # if not is_safe_prompt(question) or not reinforce_role(question):
    #     return "🚫 Uygun olmayan bir içerik algılandı."

    output = generateAnswer(RAG_LLM, chroma_collection, question)

    # if not is_safe_output(output):
    #     return "🚫 Cevap güvenlik politikalarımıza uymamaktadır."

    return output

# Gradio UI
RAG_LLM.history.clear()

def generateAnswerInterFace(question):
    return full_guardrail_interface(question)

def get_info_text():
    return "INFO: " + summarize_collection(chroma_collection)

with gr.Blocks() as demo:
    gr.Markdown("## 🎓 Üstün Yetenekli Çocuklar İçin Sosyal Gelişim Asistanı")
    query_txt = gr.Textbox(label="Enter your question here:", placeholder="Bir soru yazın...", lines=3)
    answer_txt = gr.Textbox(label="Answer:", placeholder="Answer will be displayed here", value="👋 Merhaba! Size nasıl yardımcı olabilirim?", lines=15)
    btn = gr.Button("Generate Answer")
    info_txt = gr.Textbox(get_info_text(), label="Info")
    btn.click(fn=generateAnswerInterFace, inputs=query_txt, outputs=answer_txt)

demo.launch(debug=True)
